In [ ]:
import os
import time
import random

import pandas as pd
import numpy as np
from PIL import Image
import PIL
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import load_model

NUM_FEATURES = 1
NUM_CLASSES = 2 * NUM_FEATURES

HOST_NAME = "gpu-host"
VERSION_STR = "1.0"

n_epochs = 10000
batch_size = 128
IMAGE_SHAPE = 128 # Images will be reshaped to IMAGE_SHAPE x IMAGE_SHAPE
NUM_PIXEL_CHANNELS = 3
INPUT_SHAPE = (IMAGE_SHAPE, IMAGE_SHAPE, NUM_PIXEL_CHANNELS)
LATENT_DIM = 100


if False:
    # Set seeds for reproducibility
    SEED = 2024
    random.seed(SEED)
    np.random.seed(SEED)
    tf.random.set_seed(SEED)


gpus = tf.config.experimental.list_physical_devices("GPU")
if len(gpus) > 0:
    tf.config.experimental.set_memory_growth(gpus[0], True)
    print(f"{len(tf.config.list_physical_devices('GPU'))} GPU(s) available.")
else:
    print("No GPU(s) available.")

PLOT_SETTINGS = {"text.usetex": True,
                 "font.family": "serif",
                 "figure.figsize": (8.0, 6.0),
                 "font.size": 16,
                 "axes.labelsize": 16,
                 "legend.fontsize": 14,
                 "xtick.labelsize": 14,
                 "ytick.labelsize": 14,
                 "axes.titlesize": 24,
                 "lines.linewidth": 2.0,
                 }
plt.rcParams.update(PLOT_SETTINGS)

In [ ]:
class CelebADataLoader(keras.utils.Sequence):
    """
        Data loader based on the given data loader for
        assignment 1 but modified to load the CelebA data set.
    """
    def __init__(self,
                 data_path,
                 labels_path,
                 batch_size=32,
                 num_images = 1000,
                 cache=True,
                 random_state=None,
                 dtype=np.uint8,
                 ):

        self.data_path = data_path
        self.labels_path = labels_path
        self.batch_size = max(1, int(batch_size))
        self.cache = bool(cache)
        if random_state is None:
            self.random_state = np.random
        elif isinstance(random_state, np.random.RandomState):
            self.random_state = random_state
        else:
            self.random_state = np.random.RandomState(random_state)
        self.dtype = dtype

        if self.data_path is None:
            raise ValueError('The data path is not defined.')

        if not os.path.isdir(self.data_path):
            raise ValueError('The data path is incorrectly defined.')

        self.labels_df = pd.read_csv(self.labels_path)
        self.labels_df = self.labels_df[['image_id', 'Male']]
        self._file_idx = 0
        self._images = self.labels_df['image_id'].tolist() # Get all image names in list
        self._labels = self.labels_df.drop(columns=['image_id']).values
        female_indices = self._labels == -1
        self._labels[female_indices] = 0
        if len(self._images) > num_images:
            self._images = self._images[:num_images]
            self._labels = self._labels[:num_images]

        self._image_cache = dict()
        self.on_epoch_end()

    def __len__(self):
        """Get the number of mini-batches per epoch."""
        return int(np.ceil(len(self._images) / self.batch_size))

    def __getitem__(self, index):
        """Get one batch of data."""
        # Generate indices of the batch
        indices = self._indices[index * self.batch_size:(index + 1) * self.batch_size]

        # Find the next set of file indices
        minibatch_files = [self._images[k] for k in indices]
        minibatch_labels = [self._labels[k] for k in indices]

        # Load up the corresponding minibatch
        minibatch = self.__load_minibatch(minibatch_files)

        return np.array(minibatch), np.array(minibatch_labels)

    def on_epoch_end(self):
        """Update indices after each epoch."""
        self._indices = np.arange(len(self._images))
        self.random_state.shuffle(self._indices)

    def __load_image(self, file):
        """Load a single image from file."""
        im = Image.open(os.path.join(self.data_path, file))
        if im.mode != "RGB":
            im = im.convert("RGB")
        
        im = im.resize((IMAGE_SHAPE, IMAGE_SHAPE), PIL.Image.Resampling.LANCZOS)
        im = (np.asarray(im, dtype=self.dtype) / 255.0) * 2.0 - 1.0
        
        return im

    def __load_minibatch(self, minibatch_files):
        """Load the next minibatch of samples."""

        minibatch = [None] * len(minibatch_files)
        for i, file in enumerate(minibatch_files):
            if self.cache:
                if file in self._image_cache:
                    im = self._image_cache[file]
                else:
                    im = self.__load_image(file)
                    self._image_cache[file] = im
            else:
                im = self.__load_image(file)

            minibatch[i] = im

        return minibatch

# Example usage:
data_dir = "./img_align_celeba/img_align_celeba/"
labels_path = "./list_attr_celeba.csv"

# Create the data loaders
train_ds = CelebADataLoader(data_dir, labels_path, num_images = 1024*40, batch_size=batch_size)

# A quick summary of the data:
print(f"Number of training mini-batches: {len(train_ds)}")
print(f"Number of training images      : {len(train_ds._indices)}")
print(f"\nImage shape: {np.array(train_ds[0][0][0]).shape}")

In [ ]:
# Number of images to plot
num_images = 16

# Get a batch of images and labels
images, labels = train_ds[0]  # Get the first batch, adjust index if needed

selected_images = images[:num_images]
selected_labels = labels[:num_images]

fig, axes = plt.subplots(4, 4, figsize=(10, 10))
axes = axes.flatten()

for img, ax in zip(selected_images, axes):
    ax.imshow(((img + 1) / 2 * 255).astype(np.uint8))  # Rescale image back to [0, 255] and convert to uint8
    ax.axis('off')

plt.tight_layout()
plt.show()

In [ ]:

def discriminator():
    in_label = tf.keras.layers.Input(shape=(NUM_FEATURES,))
    li = tf.keras.layers.Embedding(NUM_CLASSES, 50)(in_label)
    li = tf.keras.layers.Dense(IMAGE_SHAPE * IMAGE_SHAPE * NUM_PIXEL_CHANNELS)(li)
    li = tf.keras.layers.Reshape(INPUT_SHAPE)(li)
    
    in_image = tf.keras.layers.Input(shape=INPUT_SHAPE)
    merge = tf.keras.layers.Concatenate()([in_image, li])

    x = tf.keras.layers.Conv2D(64, (3,3), strides=(2,2), padding='same')(merge)
    x = tf.keras.layers.LeakyReLU(alpha=0.2)(x)

    x = tf.keras.layers.Conv2D(128, (3,3), strides=(2,2), padding='same')(x)
    x = tf.keras.layers.LeakyReLU(alpha=0.2)(x)
    
    x = tf.keras.layers.Conv2D(256, (3,3), strides=(2,2), padding='same')(x)
    x = tf.keras.layers.LeakyReLU(alpha=0.2)(x)
    
    x = tf.keras.layers.Conv2D(512, (3,3), strides=(2,2), padding='same')(x)
    x = tf.keras.layers.LeakyReLU(alpha=0.2)(x)
    
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    out_layer = tf.keras.layers.Dense(1, activation='sigmoid')(x)
    
    model = tf.keras.models.Model([in_image, in_label], out_layer)
    model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5), metrics=['accuracy'])
    
    return model


def generator():
    in_label = tf.keras.layers.Input(shape=(NUM_FEATURES,))

    foundation_dim = 8
    li = tf.keras.layers.Embedding(NUM_CLASSES, 50)(in_label)
    li = tf.keras.layers.Dense(foundation_dim*foundation_dim*NUM_PIXEL_CHANNELS)(li)
    li = tf.keras.layers.Reshape((foundation_dim, foundation_dim, NUM_PIXEL_CHANNELS))(li)
    
    in_lat = tf.keras.layers.Input(shape=(LATENT_DIM,))
    
    # foundation for 8x8
    x = tf.keras.layers.Dense(128 * foundation_dim * foundation_dim)(in_lat)
    x = tf.keras.layers.LeakyReLU(alpha=0.2)(x)
    x = tf.keras.layers.Reshape((foundation_dim, foundation_dim, 128))(x)

    merge = tf.keras.layers.Concatenate()([x, li])

    # upsample to 16x16
    x = tf.keras.layers.Conv2DTranspose(1024, (5,5), strides=(2,2), padding='same')(merge)
    x = tf.keras.layers.LeakyReLU(alpha=0.2)(x)

    # upsample to 32x32
    x = tf.keras.layers.Conv2DTranspose(512, (5,5), strides=(2,2), padding='same')(x)
    x = tf.keras.layers.LeakyReLU(alpha=0.2)(x)

    # upsample to 64x64
    x = tf.keras.layers.Conv2DTranspose(256, (5,5), strides=(2,2), padding='same')(x)
    x = tf.keras.layers.LeakyReLU(alpha=0.2)(x)
    
    # upsample to 128x128
    x = tf.keras.layers.Conv2DTranspose(128, (5,5), strides=(2,2), padding='same')(x)
    x = tf.keras.layers.LeakyReLU(alpha=0.2)(x)

    out_layer = tf.keras.layers.Conv2DTranspose(NUM_PIXEL_CHANNELS, (5,5), activation='tanh', padding='same')(x)
    model = tf.keras.models.Model([in_lat, in_label], out_layer)
    
    return model


def gan(generator_model, discriminator_model):
    discriminator_model.trainable = False # freez discirminator

    # get noise and label inputs from generator model
    gen_noise, gen_label = generator_model.input
    # get image output from the generator model
    gen_output = generator_model.output

    gan_output = discriminator_model([gen_output, gen_label])
    
    model = tf.keras.models.Model([gen_noise, gen_label], gan_output)
    model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5))

    return model


def get_latent_batch(number_of_samples):
    return [np.random.randn(number_of_samples, LATENT_DIM).reshape(number_of_samples, LATENT_DIM), np.random.choice([0, 1], size=number_of_samples)]


def generate_fake_batch(generator_model, number_of_samples):
    x_input, x_labels = get_latent_batch(number_of_samples)
    X = generator_model.predict([x_input, x_labels])
    y = np.zeros((number_of_samples, 1))
    return [X, x_labels], y


def save_plot(images, epoch, n=8):
    plt.figure(figsize=(30, 30))
    for i in range(n * n):
        plt.subplot(n, n, 1 + i)
        plt.axis('off')
        image = images[i, :, :, :]
        image = (image + 1) / 2.0
        plt.imshow(image)
    # save plot to file
    file_path = f"./imgs/{VERSION_STR}_{HOST_NAME}_cGAN_generated_plot_{epoch:03d}.png"
    plt.savefig(file_path)
    plt.close()
 

def summarize_performance(epoch, gen_model, disc_model, gan_model, X_real, y_real):
    _, acc_real = disc_model.evaluate([np.array(X_real), np.array(y_real)], np.ones((batch_size, 1)), verbose=0)
    
    [X_fake, X_fake_labels], y_fake = generate_fake_batch(gen_model, batch_size)

    _, acc_fake = disc_model.evaluate([X_fake, X_fake_labels], y_fake, verbose=0)
    
    print(f"Accuracy real: {acc_real*100:.0f}%, fake: {acc_fake*100:.0f}%")
    
    start_epoch = 200
    save_plot(X_fake, epoch+start_epoch)
    if True:
        # Save model and waights
        gen_model.save(f"./models/{VERSION_STR}_{HOST_NAME}_cGAN_generator_model_{epoch+start_epoch}.h5")
        
        gen_model.save_weights(f"./checkpoints/{VERSION_STR}_{HOST_NAME}_cGAN_generator_checkpoint_{epoch+start_epoch}.h5")
        disc_model.save_weights(f"./checkpoints/{VERSION_STR}_{HOST_NAME}_cGAN_discriminator_checkpoint_{epoch+start_epoch}.h5")
        gan_model.save_weights(f"./checkpoints/{VERSION_STR}_{HOST_NAME}_cGAN_gan_checkpoint_{epoch+start_epoch}.h5")


################################################################################################
########################################### TRAINING ###########################################
################################################################################################


VERBOSE_LEVEL = 1
LOAD_MODEL = False
SAVE_INTERVAL = 1


gen_model = generator()
disc_model = discriminator()
gan_model = gan(gen_model, disc_model)

if LOAD_MODEL:
    try:
        checkpoint_num = 12
        gen_model.load_weights(f"./checkpoints/{VERSION_STR}_{HOST_NAME}_cGAN_generator_checkpoint_{checkpoint_num}.h5")
        disc_model.load_weights(f"./checkpoints/{VERSION_STR}_{HOST_NAME}_cGAN_discriminator_checkpoint_{checkpoint_num}.h5")
        gan_model.load_weights(f"./checkpoints/{VERSION_STR}_{HOST_NAME}_cGAN_gan_checkpoint_{checkpoint_num}.h5")
        print("Weights loaded successfully")
    except Exception as e:
        print("Error loading weights:", e)

if VERBOSE_LEVEL > 0:
    gen_model.summary()
    disc_model.summary()
    gan_model.summary()


batch_per_epoch = len(train_ds)
for epoch in range(n_epochs):
    for batch_number, (X_real, y_real) in enumerate(train_ds):
        # Train discriminator on real images
        disc_loss_real, _ = disc_model.train_on_batch([np.array(X_real), np.array(y_real)], np.ones((batch_size, 1)))
        # Train discriminator on fake images
        [X_fake, X_fake_labels], y_fake = generate_fake_batch(gen_model, batch_size)
        d_loss_fake, _ = disc_model.train_on_batch([np.array(X_fake), np.array(X_fake_labels)], np.zeros((batch_size, 1)))
        
        # Train cGAN model with latent points
        [X_gan, X_gan_labels] = get_latent_batch(batch_size)
        g_loss = gan_model.train_on_batch([X_gan, X_gan_labels], np.ones((batch_size, 1)))
    
        print(f"Epoch: {epoch+1}/{n_epochs}, Batch: {batch_number+1}/{batch_per_epoch}, Discriminator loss: {(disc_loss_real+d_loss_fake)*0.5}, Generator loss: {g_loss}")
    
    train_ds.on_epoch_end()

    if (epoch+1) % SAVE_INTERVAL == 0:
        summarize_performance(epoch+1, gen_model, disc_model, gan_model, X_real, y_real)

In [ ]:
model_path = "./models/1.0_gpu-host_cGAN_generator_model_259.h5"
gen_model = load_model(model_path)


def plot_batch(images, n):
    plt.figure(figsize=(30, 30))
    for i in range(n):
        plt.subplot(1, n, 1 + i)
        plt.axis('off')
        image = images[i, :, :, :]
        image = (image + 1) / 2.0
        plt.imshow(image)
    plt.show()  # Display the plot

num_images = 3
x_input, x_labels = get_latent_batch(num_images)
x_labels[0] = 0

X = gen_model.predict([x_input, x_labels])
plot_batch(X, num_images)

gen_model = None